In [7]:
from dotenv import load_dotenv
import os

load_dotenv('/Users/p.miroshin/Documents/Masters/2d_year/keys.env')

ACCESS_SECRET_KEY = os.getenv("ACCESS_SECRET_KEY")
ACCESS_KEY_ID = os.getenv("ACCESS_KEY_ID")

In [8]:
import boto3

session = boto3.session.Session()
s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    region_name='ru-central1',
    aws_access_key_id=os.environ.get('ACCESS_KEY_ID'),
    aws_secret_access_key=os.environ.get('ACCESS_SECRET_KEY')
)

BUCKET_NAME = 'bucket-first'

def upload_empty_csv(file_name, file_key):
    s3.put_object(Bucket=BUCKET_NAME, Key=file_key, Body=file_name)
    print(f"{file_key} uploaded to bucket {BUCKET_NAME}.")

In [9]:
users_csv = "chat_id,shikimori_username,last_updated\n"
anime_csv = "anime_id,title,average_score,genres,type,episode_duration,episode_count,poster_url\n"
user_anime_csv = "user_anime_id,chat_id,anime_id,status,user_score\n"
recommendations_csv = "recommendation_id,user_id,anime_id,date_recommended,user_score\n"
logs_csv = "log_id,user_id,anime_id,action,details,timestamp\n"

In [10]:
upload_empty_csv(users_csv, 'users.csv')
upload_empty_csv(anime_csv, 'anime.csv')
upload_empty_csv(user_anime_csv, 'user_anime.csv')
upload_empty_csv(recommendations_csv, 'recommendations.csv')
upload_empty_csv(logs_csv, 'logs.csv')

users.csv uploaded to bucket bucket-first.
anime.csv uploaded to bucket bucket-first.
user_anime.csv uploaded to bucket bucket-first.
recommendations.csv uploaded to bucket bucket-first.
logs.csv uploaded to bucket bucket-first.
